In [1]:
import numpy as np
import keras
import keras.backend as K
from keras.models import Sequential
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, ZeroPadding2D, UpSampling2D
from keras.layers.normalization import BatchNormalization
from PIL import Image
import random
import math
import matplotlib.pyplot as plt
import dendropy
import csv
from dendropy.calculate import treecompare
import sys
import numpy
import pickle
from sklearn.model_selection import train_test_split
from keras.optimizers import RMSprop
numpy.set_printoptions(threshold=sys.maxsize)

Using TensorFlow backend.


In [2]:
from sklearn.model_selection import train_test_split

In [3]:
sys.path.append("../..")

In [4]:
import Bandelt_Encode_Decode.Bandelt_Encode_func as BN_Encode_func
import Bandelt_Encode_Decode.Bandelt_Decode_func as BN_Decode_func
import Bandelt_Encode_Decode.Bandelt_Node as BN

# Loading Data

In [5]:
X_Input_Alignment_Data = np.load('../../ALL_DATA/499_dataset_Input_Output/X_Input_Alignment_Data.npy')

In [6]:
print(X_Input_Alignment_Data.shape)

(499, 100, 1000, 5)


In [50]:
for index, sample in enumerate(X_Input_Alignment_Data[0]):
    print('Sample ', index, ': ', )
    for sequence in sample:
#         print("A: ", sequence.astype(int))
#         print("B: ", np.array([0,0,0,0,0]))
        if np.array_equal(sequence.astype(int), np.array([0,0,0,0,0])):
            print('_', end='')
        elif np.array_equal(sequence.astype(int), np.array([1,0,0,0,0])):
            print('A', end='')
        elif np.array_equal(sequence.astype(int), np.array([0,1,0,0,0])):
            print('T', end='')
        elif np.array_equal(sequence.astype(int), np.array([0,0,1,0,0])):
            print('C', end='')
        elif np.array_equal(sequence.astype(int), np.array([0,0,0,1,0])):
            print('G', end='')
        elif np.array_equal(sequence.astype(int), np.array([0,0,0,0,1])):
            print('X', end='')
    print('\n')

Sample  0 : 
___________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

C________CC___________________C__________________G__C_____________________________G______________________CC____________________________CGGT__________G____________CT________________A___________________________G__________CC__________G__________GC__T_________________T________________T______________G______G_________TC__C______T____G___________C_____C__________________G_____T__G_____C_T__G______T__________________G_________________________C____G_______C_____________________________________T____G_____C________G_________A______G________G_____________________________________________________________C________________CG_____________T____C____T_CG___________________GCG_____________C_C__T_______T_______G_G_C__C_________________C______C__C_______________G____________CC______C_G____C___C____________________C__C__________________________T_____________G__G_____G___________________________G__TCGC_____________________G__CC_____C_C______________G____________________G____________C______C__C________________

C________CC___________________G__________________G__C_____________________________C______________________CC____________________________CGGT__________G____________CC________________A___________________________G__________CC__________G__________GC__C_________________C________________C______________G______G_________GC__C______C____C___________C_____C__________________G_____G__G_____C_C__G______T__________________G_________________________C____G_______C_____________________________________T____A_____C________C_________A______G________G_____________________________________________________________C________________CG_____________T____C____T_CG___________________ACG_____________C_G__T_______A_______C_A_C__C_________________C______C__C_______________G____________CA______C_G____C___C____________________C__C__________________________T_____________G__G_____G___________________________G__TCGC_____________________G__CC_____C_C______________G____________________G____________C______C__C________________

________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

C________CC___________________C__________________G__T_____________________________C______________________CC____________________________CGGT__________G____________CT________________A___________________________G__________CC__________G__________AC__T_________________G________________C______________A______A_________GA__C______T____G___________C_____C__________________G_____C__G_____C_X__G______T__________________G_________________________C____G_______C_____________________________________T____G_____C________G_________A______G________A_____________________________________________________________C________________CG_____________T____X____C_AG___________________ACG_____________C_C__T_______C_______G_T_C__C_________________C______C__C_______________G____________CC______C_G____C___C____________________C__C__________________________T_____________G__G_____G___________________________G__TTTC_____________________G__CC_____C_C______________T____________________G____________C______A__C________________

...

C________CC___________________C__________________A__C_____________________________G______________________CC____________________________CGAT__________G____________CT________________G___________________________G__________CG__________G__________GC__C_________________T________________T______________A______A_________GC__C______T____G___________C_____C__________________G_____C__G_____C_T__A______T__________________G_________________________G____G_______C_____________________________________T____A_____A________C_________A______G________G_____________________________________________________________C________________CG_____________T____C____C_TC___________________ACG_____________C_C__T_______T_______G_C_C__C_________________C______C__C_______________G____________CC______C_G____C___C____________________G__C__________________________T_____________G__G_____G___________________________G__CCGC_____________________A__CC_____C_C______________C____________________G____________C______C__C________________

A________CT___________________C__________________G__C_____________________________G______________________CC____________________________CAGG__________G____________CC________________A___________________________G__________CC__________G__________GC__C_________________T________________C______________A______A_________CC__C______T____G___________T_____C__________________G_____C__G_____C_T__G______T__________________G_________________________C____G_______C_____________________________________T____A_____A________G_________A______G________G_____________________________________________________________C________________CG_____________T____T____C_AG___________________GCG_____________C_C__T_______A_______G_C_C__C_________________C______C__C_______________G____________CC______C_G____C___C____________________C__C__________________________T_____________A__G_____G___________________________G__TTGC_____________________A__CC_____C_C______________C____________________G____________C______A__C________________

C________CT___________________C__________________G__C_____________________________C______________________CC____________________________CGGT__________G____________CT________________A___________________________G__________CC__________G__________GC__C_________________C________________C______________A______G_________CA__C______A____G___________C_____C__________________G_____C__G_____C_T__G______T__________________G_________________________C____G_______C_____________________________________C____A_____C________G_________A______G________G_____________________________________________________________C________________CG_____________T____T____C_CG___________________ACG_____________C_C__T_______A_______G_C_T__C_________________C______C__C_______________G____________CC______C_G____C___C____________________C__C__________________________T_____________A__G_____G___________________________G__TCGC_____________________A__CC_____C_C______________C____________________G____________C______C__C________________

C________CT___________________C__________________G__C_____________________________C______________________CC____________________________CGGT__________G____________CT________________A___________________________G__________CC__________G__________GC__T_________________G________________C______________G______A_________AA__C______C____G___________C_____C__________________G_____C__G_____C_T__G______T__________________G_________________________C____A_______C_____________________________________T____A_____C________G_________A______G________G_____________________________________________________________C________________CG_____________T____T____C_TG___________________ACG_____________C_C__T_______T_______C_C_C__C_________________C______C__C_______________G____________CC______C_G____C___C____________________C__C__________________________T_____________A__G_____G___________________________G__TCTC_____________________G__CC_____G_C______________C____________________G____________C______G__C________________

________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

In [7]:
train_X,valid_X,train_ground,valid_ground = train_test_split(X_Input_Alignment_Data,
                                                             X_Input_Alignment_Data, 
                                                             test_size=0.2, 
                                                             random_state=13)

## Shuffle Input Data

In [8]:
for i in range(X_Input_Alignment_Data.shape[0]):
    shuffle_indices_100 = random.sample(range(0, 100), 100)
    shuffle_indices_1000 = random.sample(range(0, 1000), 1000)
    tmp = X_Input_Alignment_Data[i][shuffle_indices_100]
    tmp = tmp[:, shuffle_indices_1000]
    X_Input_Alignment_Data[i] = tmp

## Parameters Initialization

In [9]:
batch_size = 128
epochs = 50
inChannel = 5
BRANCH_NUM = 100
SEQUENCE_LEN = 1000
input_img = Input(shape = (BRANCH_NUM, SEQUENCE_LEN, inChannel))

In [10]:
X_Input_Alignment_Data[0][0]

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0.

In [11]:
def autoencoder(input_img):
    #encoder
    #input = 28 x 28 x 1 (wide and thin)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img) #100 x 1000 x 32
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1) #14 x 14 x 32
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1) #50 x 500 x 64
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2) #7 x 7 x 64
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2) #25 x 250 x 128 (small and thick)

    #decoder
    conv4 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3) #25 x 250 x 128
    up1 = UpSampling2D((2,2))(conv4) # 14 x 14 x 128
    conv5 = Conv2D(64, (3, 3), activation='relu', padding='same')(up1) # 50 x 500 x 64
    up2 = UpSampling2D((2,2))(conv5) # 28 x 28 x 64
    decoded = Conv2D(5, (3, 3), activation='sigmoid', padding='same')(up2) # 100 x 1000 x 1
    return decoded

In [12]:
autoencoder = Model(input_img, autoencoder(input_img))
autoencoder.compile(loss='mean_squared_error', optimizer = RMSprop())

In [13]:
autoencoder.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100, 1000, 5)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 100, 1000, 32)     1472      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 500, 32)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 500, 64)       18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 25, 250, 64)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 25, 250, 128)      73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 25, 250, 128)      1475

In [14]:
autoencoder_train = autoencoder.fit(train_X, train_ground, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(valid_X, valid_ground))

Train on 399 samples, validate on 100 samples
Epoch 1/50
399/399 [==============================] - 125s 313ms/step - loss: 0.1949 - val_loss: 0.0188
Epoch 2/50
399/399 [==============================] - 121s 303ms/step - loss: 0.0192 - val_loss: 0.0182
Epoch 3/50


KeyboardInterrupt: 

In [53]:
autoencoder.save('./output/autoencoder_model.h5')

## Start Building Model

### Load Data